<h2>Working with GRIB2 Data on Amazon AWS Using <i>xarray</i>, <i>cfgrib</i>, and <i>fsspec</i></h2>

<i>We gratefully acknowledge the guidance provided in <a href="https://stackoverflow.com/questions/66229140/xarray-read-remote-grib-file-on-s3-using-cfgrib"><b>this StackOverflow thread</b></a> on using xarray to read GRIB2 files stored on Amazon AWS s3 using cfgrib and xarray.</i>

This notebook demonstrates how the <i>cfgrib</i> engine (local version: 0.9.8.5) to <i>xarray</i> (local version: 0.19.0; also requires that dask also be installed by virtue of open multiple files at once), provided by the <i>eccodes</i> package (local version: 2.23.0), can be used to work with Global Ensemble Forecast System (GEFS) GRIB2 data downloaded from an Amazon AWS bucket using <i>fsspec</i> (local version: 2021.9.0).

In this example, we begin by loading the xarray and fsspec modules. We do not need to separately load the cfgrib or eccodes modules, as they can be accessed directly through xarray.

In [1]:
import xarray as xr
import fsspec

For this example, we wish to access the 6-h forecasts from all 30 GEFS members from the 30 September 2021 1800 UTC forecast cycle. We work with the pgrb2ap5 files, where the "a" indicates that these files contain the most-common variables and the "p5" indicates that they are at 0.5 deg horizontal grid spacing. These data are available in the s3 noaa-gefs-pds Amazon AWS bucket, with more information available at https://registry.opendata.aws/noaa-gefs/. 

We start by creating a list that contains the paths to each of the 30 GEFS members' 6-h forecasts from this forecast cycle. The "simplecache::" option preceding each file name is an fsspec option that specifies how data should be locally saved (in this case, persistently within the directory specified later when we access the data). We then download and cache the files locally (noting that these s3 data can be accessed anonymously) in /tmp/files. Finally, we attempt to open the data as an xarray multiple-file dataset using the cfgrib engine. Each ensemble member contains a coordinate dimension named <i>number</i> that specifies the ensemble member number (from 1 to 30). We leverage this information to tell xarray to concatenate the data along this dimension.

In [2]:
# Create a list of all 30 GEFS members.
filelist = []
for i in range(1,31):
    if(i < 10):
        filelist.append("simplecache::s3://noaa-gefs-pds/gefs.20210930/18/atmos/pgrb2ap5/gep0"+str(i)+".t18z.pgrb2a.0p50.f006")
    else:
        filelist.append("simplecache::s3://noaa-gefs-pds/gefs.20210930/18/atmos/pgrb2ap5/gep"+str(i)+".t18z.pgrb2a.0p50.f006")

# Use fsspec to cache these files locally, then
# open them all at once using xarray/cfgrib.
gribfiles = fsspec.open_local(filelist, s3={'anon': True}, filecache={'cache_storage':'/tmp/files'})
data = xr.open_mfdataset(gribfiles, combine="nested", concat_dim="number", engine="cfgrib")

skipping variable: paramId==130 shortName='t'
Traceback (most recent call last):
  File "/opt/tljh/user/lib/python3.7/site-packages/cfgrib/dataset.py", line 602, in build_dataset_components
    dict_merge(variables, coord_vars)
  File "/opt/tljh/user/lib/python3.7/site-packages/cfgrib/dataset.py", line 538, in dict_merge
    "key=%r value=%r new_value=%r" % (key, master[key], value)
cfgrib.dataset.DatasetBuildError: key present and new value is different: key='isobaricInhPa' value=Variable(dimensions=('isobaricInhPa',), data=array([1000,  925,  850,  700,  500,  300,  250,  200,  100,   50,   10])) new_value=Variable(dimensions=('isobaricInhPa',), data=array([1000,  925,  850,  700,  500,  250,  200,  100,   50,   10]))
skipping variable: paramId==157 shortName='r'
Traceback (most recent call last):
  File "/opt/tljh/user/lib/python3.7/site-packages/cfgrib/dataset.py", line 602, in build_dataset_components
    dict_merge(variables, coord_vars)
  File "/opt/tljh/user/lib/python3.7/site-

DatasetBuildError: multiple values for unique key, try re-open the file with one of:
    filter_by_keys={'typeOfLevel': 'isobaricInhPa'}
    filter_by_keys={'typeOfLevel': 'surface'}
    filter_by_keys={'typeOfLevel': 'depthBelowLandLayer'}
    filter_by_keys={'typeOfLevel': 'heightAboveGround'}
    filter_by_keys={'typeOfLevel': 'atmosphereSingleLayer'}
    filter_by_keys={'typeOfLevel': 'atmosphere'}
    filter_by_keys={'typeOfLevel': 'nominalTop'}
    filter_by_keys={'typeOfLevel': 'pressureFromGroundLayer'}
    filter_by_keys={'typeOfLevel': 'meanSea'}

It takes a minute or two to download all of the data, but as it does so, we find that the open_mfdataset call we used is not sufficient to open these files. There are a lot of warnings (shaded in pink) followed by a DatasetBuildError (shaded in white) indicating that the file contains multiple values for the key 'typeOfLevel,' meaning that this file contains data on multiple different vertical level types. The very last portion of the error message is helpful; it tells us what types of levels this file contains - in this case, nine different types! It also tells us what argument to include with our open_mfdataset command to a subset of the file, here being all of the variables on a single level type.

With this in mind, let's modify our open_mfdataset call try to open the surface data:

In [3]:
data_sfc = xr.open_mfdataset(gribfiles, combine="nested", concat_dim="number", engine="cfgrib", filter_by_keys={'typeOfLevel': 'surface'})

It takes a few seconds to open the data, but it is successful! Let's look at what we have:

In [4]:
print(data_sfc)

<xarray.Dataset>
Dimensions:     (number: 30, latitude: 361, longitude: 720)
Coordinates:
  * number      (number) int64 1 2 3 4 5 6 7 8 9 ... 22 23 24 25 26 27 28 29 30
    time        datetime64[ns] 2021-09-30T18:00:00
    step        timedelta64[ns] 06:00:00
    surface     int64 0
  * latitude    (latitude) float64 90.0 89.5 89.0 88.5 ... -89.0 -89.5 -90.0
  * longitude   (longitude) float64 0.0 0.5 1.0 1.5 ... 358.0 358.5 359.0 359.5
    valid_time  datetime64[ns] 2021-10-01
Data variables: (12/15)
    sp          (number, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    sdwe        (number, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    sde         (number, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    icetk       (number, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    tp          (number, latitude, longitude) float32 dask.arra

We have data with three varying dimensions: number (the ensemble member number, from 1 to 30), latitude, and longitude. There are also 15 variables, of which 12 are shown. We also get some metadata. Let's take a look at all of the variables:

In [5]:
print(data_sfc.variables)

Frozen({'number': <xarray.IndexVariable 'number' (number: 30)>
array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30])
Attributes:
    long_name:      ensemble member numerical id
    units:          1
    standard_name:  realization, 'time': <xarray.Variable ()>
array('2021-09-30T18:00:00.000000000', dtype='datetime64[ns]')
Attributes:
    long_name:      initial time of forecast
    standard_name:  forecast_reference_time, 'step': <xarray.Variable ()>
array(21600000000000, dtype='timedelta64[ns]')
Attributes:
    long_name:      time since forecast_reference_time
    standard_name:  forecast_period, 'surface': <xarray.Variable ()>
array(0)
Attributes:
    long_name:  original GRIB coordinate for key: level(surface)
    units:      1, 'latitude': <xarray.IndexVariable 'latitude' (latitude: 361)>
array([ 90. ,  89.5,  89. , ..., -89. , -89.5, -90. ])
Attributes:
    units:             degrees_north
    stand

There is quite a lot of information in the output above. It's partially organized - we get information about the dataset's dimension variables (number, time, step, surface, latitude, longitude, and valid_time) first, followed by information about the data variables, starting with sp (which we are told is the surface pressure). In all, we are provided with partial information about fifteen variables: sp, sdwe, sde, icetk, tp, csnow, cicep, cfrzr, crain, lhtfl, shtfl, dswrf, dlwrf, uswrf, and ulwrf. 

Note, though, that not all of the attributes for each variable print to the screen; there are just too many for each one. xarray stores these attributes for each variable in a dictionary called attrs. Let's access these attributes for the sp (surface pressure) variable:

In [6]:
print(data_sfc['sp'].attrs)

{'GRIB_paramId': 134, 'GRIB_shortName': 'sp', 'GRIB_units': 'Pa', 'GRIB_name': 'Surface pressure', 'GRIB_cfName': 'surface_air_pressure', 'GRIB_cfVarName': 'sp', 'GRIB_dataType': 'pf', 'GRIB_missingValue': 9999, 'GRIB_numberOfPoints': 259920, 'GRIB_totalNumber': 30, 'GRIB_typeOfLevel': 'surface', 'GRIB_NV': 0, 'GRIB_stepUnits': 1, 'GRIB_stepType': 'instant', 'GRIB_gridType': 'regular_ll', 'GRIB_gridDefinitionDescription': 'Latitude/longitude. Also called equidistant cylindrical, or Plate Carree', 'GRIB_Nx': 720, 'GRIB_iDirectionIncrementInDegrees': 0.5, 'GRIB_iScansNegatively': 0, 'GRIB_longitudeOfFirstGridPointInDegrees': 0.0, 'GRIB_longitudeOfLastGridPointInDegrees': 359.5, 'GRIB_Ny': 361, 'GRIB_jDirectionIncrementInDegrees': 0.5, 'GRIB_jPointsAreConsecutive': 0, 'GRIB_jScansPositively': 0, 'GRIB_latitudeOfFirstGridPointInDegrees': 90.0, 'GRIB_latitudeOfLastGridPointInDegrees': -90.0, 'long_name': 'Surface pressure', 'units': 'Pa', 'standard_name': 'surface_air_pressure'}


We reference the variable inside of brackets and the attrs dictionary of attributes with .attrs appended to the end. Some of the attributes that we did not see before are helpful for us to know if we were to want to plot these data on a map - e.g., the GRIB_gridType and GRIB_gridDefinition_Description attributes indicate that the data are on a regular latitude-longitude grid.

Let's be honest, though: we aren't likely to want to work with most of these variables. Instead, we're likely to want to work with data on the isobaricInhPa (pressure levels), heightAboveGround (e.g., 2-m and 10-m above ground level), and meanSea (mean sea-level pressure data) vertical grids. Though we can only open one level type per open_mfdataset statement, there is no limit (apart from computer memory, I suppose) on the number of different level types we can open using different open_mfdataset statements. With that in mind, let's try to open the heightAboveGround data:

In [7]:
data_hlevs = xr.open_mfdataset(gribfiles, combine="nested", concat_dim="number", engine="cfgrib", filter_by_keys={'typeOfLevel': 'heightAboveGround'})

skipping variable: paramId==165 shortName='u10'
Traceback (most recent call last):
  File "/opt/tljh/user/lib/python3.7/site-packages/cfgrib/dataset.py", line 602, in build_dataset_components
    dict_merge(variables, coord_vars)
  File "/opt/tljh/user/lib/python3.7/site-packages/cfgrib/dataset.py", line 538, in dict_merge
    "key=%r value=%r new_value=%r" % (key, master[key], value)
cfgrib.dataset.DatasetBuildError: key present and new value is different: key='heightAboveGround' value=Variable(dimensions=(), data=2) new_value=Variable(dimensions=(), data=10)
skipping variable: paramId==166 shortName='v10'
Traceback (most recent call last):
  File "/opt/tljh/user/lib/python3.7/site-packages/cfgrib/dataset.py", line 602, in build_dataset_components
    dict_merge(variables, coord_vars)
  File "/opt/tljh/user/lib/python3.7/site-packages/cfgrib/dataset.py", line 538, in dict_merge
    "key=%r value=%r new_value=%r" % (key, master[key], value)
cfgrib.dataset.DatasetBuildError: key present

Oof - that's a lot of warnings. They are all nearly identical, though: the u10 and v10 variables are skipped for each of the 30 ensemble members because the "key present and new value" are different - the key present has a value of 2 and the new value is 10. We don't get any errors, though, so let's see what's in our new data_hlevs variable:

In [8]:
print(data_hlevs)

<xarray.Dataset>
Dimensions:            (number: 30, latitude: 361, longitude: 720)
Coordinates:
  * number             (number) int64 1 2 3 4 5 6 7 8 ... 24 25 26 27 28 29 30
    time               datetime64[ns] 2021-09-30T18:00:00
    step               timedelta64[ns] 06:00:00
    heightAboveGround  int64 2
  * latitude           (latitude) float64 90.0 89.5 89.0 ... -89.0 -89.5 -90.0
  * longitude          (longitude) float64 0.0 0.5 1.0 1.5 ... 358.5 359.0 359.5
    valid_time         datetime64[ns] 2021-10-01
Data variables:
    t2m                (number, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    r2                 (number, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    tmax               (number, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    tmin               (number, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
Attrib

We have four variables: t2m, r2, tmax, and tmin. The heightAboveGround coordinate variable has a value of 2. We can infer that these data are all valid a 2-m above ground level. From their names, they appear to be 2-m temperature (t2m), 2-m relative humidity (r2), and the maximum and minimum temperature over some time interval (tmax, tmin).

In all, it looks like we have two different coordinates for data_hlevs: 2 for 2-m above ground level and 10 for 10-m above ground level. We will have to read in these data separately as well, e.g.,

In [9]:
data_hlevs_2 = xr.open_mfdataset(gribfiles, combine="nested", concat_dim="number", engine="cfgrib", filter_by_keys={'typeOfLevel': 'heightAboveGround', 'level': 2})
print(data_hlevs_2)
data_hlevs_10 = xr.open_mfdataset(gribfiles, combine="nested", concat_dim="number", engine="cfgrib", filter_by_keys={'typeOfLevel': 'heightAboveGround', 'level': 10})
print(data_hlevs_10)

<xarray.Dataset>
Dimensions:            (number: 30, latitude: 361, longitude: 720)
Coordinates:
  * number             (number) int64 1 2 3 4 5 6 7 8 ... 24 25 26 27 28 29 30
    time               datetime64[ns] 2021-09-30T18:00:00
    step               timedelta64[ns] 06:00:00
    heightAboveGround  int64 2
  * latitude           (latitude) float64 90.0 89.5 89.0 ... -89.0 -89.5 -90.0
  * longitude          (longitude) float64 0.0 0.5 1.0 1.5 ... 358.5 359.0 359.5
    valid_time         datetime64[ns] 2021-10-01
Data variables:
    t2m                (number, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    r2                 (number, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    tmax               (number, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    tmin               (number, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
Attrib

How did we know to pass in a separate level keyword through filter_by_keys? eccodes comes with a command-line program named grib_ls that, when you pass it the name of a GRIB file to read, provides information as to what eccodes/cfgrib "see" in the GRIB file. In this case, passing it a locally stored version of the GEFS data returned a column headed by 'level', with values of 2 and 10 for the heightAboveGround data. To get to that point required some Google searching and subsequent working through cfgrib's GitHub issues section! 

Ultimately, though, this approach allowed us to successfully read in the 2-m and 10-m heightAboveGround fields. A similar approach can be used for variables on any given level type that may have different <i>single</i> vertical coordinate values.

What about pressure-level data? Let's try to open these fields...

In [10]:
data_plevs = xr.open_mfdataset(gribfiles, combine="nested", concat_dim="number", engine="cfgrib", filter_by_keys={'typeOfLevel': 'isobaricInhPa'})

skipping variable: paramId==130 shortName='t'
Traceback (most recent call last):
  File "/opt/tljh/user/lib/python3.7/site-packages/cfgrib/dataset.py", line 602, in build_dataset_components
    dict_merge(variables, coord_vars)
  File "/opt/tljh/user/lib/python3.7/site-packages/cfgrib/dataset.py", line 538, in dict_merge
    "key=%r value=%r new_value=%r" % (key, master[key], value)
cfgrib.dataset.DatasetBuildError: key present and new value is different: key='isobaricInhPa' value=Variable(dimensions=('isobaricInhPa',), data=array([1000,  925,  850,  700,  500,  300,  250,  200,  100,   50,   10])) new_value=Variable(dimensions=('isobaricInhPa',), data=array([1000,  925,  850,  700,  500,  250,  200,  100,   50,   10]))
skipping variable: paramId==157 shortName='r'
Traceback (most recent call last):
  File "/opt/tljh/user/lib/python3.7/site-packages/cfgrib/dataset.py", line 602, in build_dataset_components
    dict_merge(variables, coord_vars)
  File "/opt/tljh/user/lib/python3.7/site-

Oof - more of these "key present and new value is different" errors. They are more complex this time: t and r are not defined at 300 hPa whereas the "key present" field (whatever it may be at this point) does; u and v are defined on all of the same levels as are the "key present" field <i>and</i> also 400 hPa; and w is only defined at 850 hPa. What a mess! Before we proceed, though, let's see what we <i>did</i> read in so that we can identify the "key present" field:

In [11]:
print(data_plevs)

<xarray.Dataset>
Dimensions:        (number: 30, isobaricInhPa: 11, latitude: 361, longitude: 720)
Coordinates:
  * number         (number) int64 1 2 3 4 5 6 7 8 9 ... 23 24 25 26 27 28 29 30
    time           datetime64[ns] 2021-09-30T18:00:00
    step           timedelta64[ns] 06:00:00
  * isobaricInhPa  (isobaricInhPa) int64 1000 925 850 700 500 ... 200 100 50 10
  * latitude       (latitude) float64 90.0 89.5 89.0 88.5 ... -89.0 -89.5 -90.0
  * longitude      (longitude) float64 0.0 0.5 1.0 1.5 ... 358.5 359.0 359.5
    valid_time     datetime64[ns] 2021-10-01
Data variables:
    gh             (number, isobaricInhPa, latitude, longitude) float32 dask.array<chunksize=(1, 11, 361, 720), meta=np.ndarray>
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP 
    GRIB_subCentre:          2
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP 
    histo

We read in a single variable - gh, which is the geopotential height. However, we at least now know all of the variables with this isobaricInhPa grid type: gh, u, v, w, t, and r. We can use this information - which represents each variable's shortName attribute - to read in a single variable on one or all of its levels. To illustrate, let's read in the temperature variable - first at all levels, then only at 500 hPa.

In [12]:
data_plevs_t = xr.open_mfdataset(gribfiles, combine="nested", concat_dim="number", engine="cfgrib", filter_by_keys={'typeOfLevel': 'isobaricInhPa', 'shortName': 't'})
print(data_plevs_t)
data_plevs_t500 = xr.open_mfdataset(gribfiles, combine="nested", concat_dim="number", engine="cfgrib", filter_by_keys={'typeOfLevel': 'isobaricInhPa', 'shortName': 't', 'level': 500})
print(data_plevs_t500)

<xarray.Dataset>
Dimensions:        (number: 30, isobaricInhPa: 10, latitude: 361, longitude: 720)
Coordinates:
  * number         (number) int64 1 2 3 4 5 6 7 8 9 ... 23 24 25 26 27 28 29 30
    time           datetime64[ns] 2021-09-30T18:00:00
    step           timedelta64[ns] 06:00:00
  * isobaricInhPa  (isobaricInhPa) int64 1000 925 850 700 500 250 200 100 50 10
  * latitude       (latitude) float64 90.0 89.5 89.0 88.5 ... -89.0 -89.5 -90.0
  * longitude      (longitude) float64 0.0 0.5 1.0 1.5 ... 358.5 359.0 359.5
    valid_time     datetime64[ns] 2021-10-01
Data variables:
    t              (number, isobaricInhPa, latitude, longitude) float32 dask.array<chunksize=(1, 10, 361, 720), meta=np.ndarray>
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP 
    GRIB_subCentre:          2
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP 
    histo

The former varies over four dimensions: ensemble member, pressure, latitude, and longitude. The latter varies only over three dimensions: ensemble member, latitude, and longitude.

<hr>

Loading the data has been fun but is only half of the battle. Once we've loaded the data, we often want to do something with it! If it's a map we're after, cartopy can work with xarray datasets without much issue - we just want to make sure that the data vary in only two dimensions (latitude and longitude) before calling the mapping function. Maybe it's a single ensemble member that we are after. In this case, we can use xarray's <i>sel</i> function to select the desired member. Here, we select ensemble member 15 from the 500 hPa temperature data we loaded a moment ago:

In [13]:
t500_mem1 = data_plevs_t500.sel(number=15)
print(t500_mem1)

<xarray.Dataset>
Dimensions:        (latitude: 361, longitude: 720)
Coordinates:
    number         int64 15
    time           datetime64[ns] 2021-09-30T18:00:00
    step           timedelta64[ns] 06:00:00
    isobaricInhPa  int64 500
  * latitude       (latitude) float64 90.0 89.5 89.0 88.5 ... -89.0 -89.5 -90.0
  * longitude      (longitude) float64 0.0 0.5 1.0 1.5 ... 358.5 359.0 359.5
    valid_time     datetime64[ns] 2021-10-01
Data variables:
    t              (latitude, longitude) float32 dask.array<chunksize=(361, 720), meta=np.ndarray>
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP 
    GRIB_subCentre:          2
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP 
    history:                 2021-10-04T12:33:46 GRIB to CDM+CF via cfgrib-0....


Or, maybe it's the ensemble mean that we are after. xarray's <i>mean</i> function can be used to take the mean across a specified dimension. Here, we compute the ensemble-mean 500 hPa temperature:

In [14]:
t500_avg = data_plevs_t500.mean(dim='number')
print(t500_avg)

<xarray.Dataset>
Dimensions:        (latitude: 361, longitude: 720)
Coordinates:
    time           datetime64[ns] 2021-09-30T18:00:00
    step           timedelta64[ns] 06:00:00
    isobaricInhPa  int64 500
  * latitude       (latitude) float64 90.0 89.5 89.0 88.5 ... -89.0 -89.5 -90.0
  * longitude      (longitude) float64 0.0 0.5 1.0 1.5 ... 358.5 359.0 359.5
    valid_time     datetime64[ns] 2021-10-01
Data variables:
    t              (latitude, longitude) float32 dask.array<chunksize=(361, 720), meta=np.ndarray>


Note how the number coordinate has gone away by virtue of taking the mean across this coordinate, leaving us with 2-D data.

Alternatively, sometimes we may be interested in only a single location's worth of data. Since latitude and longitude are coordinate dimensions to our dataset, we can use xarray's <i>sel</i> function to parse the data down to a single location, even if that location does not have an exact match in our data's latitude and longitude array (as is often the case). For instance, Tallahassee, FL is located at 30.44°N, 84.28°W. Let's select all of the 500 hPa temperature data for Tallahassee:

In [15]:
t500_tlh = data_plevs_t500.sel(latitude=30.44, longitude=275.72, method='nearest')
print(t500_tlh)

<xarray.Dataset>
Dimensions:        (number: 30)
Coordinates:
  * number         (number) int64 1 2 3 4 5 6 7 8 9 ... 23 24 25 26 27 28 29 30
    time           datetime64[ns] 2021-09-30T18:00:00
    step           timedelta64[ns] 06:00:00
    isobaricInhPa  int64 500
    latitude       float64 30.5
    longitude      float64 275.5
    valid_time     datetime64[ns] 2021-10-01
Data variables:
    t              (number) float32 dask.array<chunksize=(1,), meta=np.ndarray>
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP 
    GRIB_subCentre:          2
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP 
    history:                 2021-10-04T12:33:46 GRIB to CDM+CF via cfgrib-0....


The latitude and longitude specifications in this statement refer to their coordinate names. Other data may refer to these differently, necessitating using those different names for those data.

Earlier print statements for our dataset indicated that the longitude data varied from 0 to 359.5 rather than -180 to 179.5. This necessitates specifying the desired longitude in this range. For Tallahassee, 84.28°W = 360°-84.28° = 275.72°, which we use as our longitude specification.

Finally, the method='nearest' option tells xarray to find and return the data from the point nearest to the specified location. In this case, Tallahassee's nearest point in our data is at 30.5°N and 275.5°=84.5°W.

We can print the actual data values from t500_tlh:

In [16]:
print(t500_tlh.t.values)

[265.22137 264.9746  264.42365 264.42136 264.73944 264.77863 264.50854
 264.24274 264.6056  264.63702 264.97864 263.63153 264.60004 264.44275
 264.68542 264.81577 263.74274 264.8427  264.4079  264.9     264.0427
 264.72137 265.2     264.84268 264.7079  264.7427  264.909   264.8
 264.93704 264.3    ]


Note the multipart specification to this print statement: the t500_tlh Python variable, then its t variable, then its values. In this example, we return 30 values for the 500 hPa temperature at Tallahassee, FL from our dataset, one per ensemble member. Based on the values we see, they appear to be in K. We can confirm this by printing the metadata for this variable. We can access this information directly by calling the variable; alternatively, as demonstrated earlier, we could explicitly specify just the attributes with t500_tlh['t'].attrs.

In [17]:
print(t500_tlh.t)

<xarray.DataArray 't' (number: 30)>
dask.array<getitem, shape=(30,), dtype=float32, chunksize=(1,), chunktype=numpy.ndarray>
Coordinates:
  * number         (number) int64 1 2 3 4 5 6 7 8 9 ... 23 24 25 26 27 28 29 30
    time           datetime64[ns] 2021-09-30T18:00:00
    step           timedelta64[ns] 06:00:00
    isobaricInhPa  int64 500
    latitude       float64 30.5
    longitude      float64 275.5
    valid_time     datetime64[ns] 2021-10-01
Attributes: (12/30)
    GRIB_paramId:                             130
    GRIB_shortName:                           t
    GRIB_units:                               K
    GRIB_name:                                Temperature
    GRIB_cfName:                              air_temperature
    GRIB_cfVarName:                           t
    ...                                       ...
    GRIB_jScansPositively:                    0
    GRIB_latitudeOfFirstGridPointInDegrees:   90.0
    GRIB_latitudeOfLastGridPointInDegrees:    -90.0
    long_n

Although not all of the attributes print, we see two attributes that indicate the data's units: GRIB_units and units both indicate that the data are in K, as expected. If for some reason we wanted to convert these data to °C, we can apply the desired formula onto the full dataset:

In [18]:
t500_tlh_degC = t500_tlh-273.15
print(t500_tlh_degC.t.values)

[-7.9286194 -8.1753845 -8.726349  -8.728638  -8.410553  -8.371368
 -8.641449  -8.907257  -8.544403  -8.51297   -8.171356  -9.518463
 -8.549957  -8.707245  -8.464569  -8.3342285 -9.407257  -8.3072815
 -8.742096  -8.25      -9.1073    -8.428619  -7.9499817 -8.307312
 -8.442108  -8.407288  -8.240997  -8.350006  -8.212952  -8.850006 ]


Doing so wipes out the variable's attributes, however:

In [19]:
print(t500_tlh_degC.t)

<xarray.DataArray 't' (number: 30)>
dask.array<sub, shape=(30,), dtype=float32, chunksize=(1,), chunktype=numpy.ndarray>
Coordinates:
  * number         (number) int64 1 2 3 4 5 6 7 8 9 ... 23 24 25 26 27 28 29 30
    time           datetime64[ns] 2021-09-30T18:00:00
    step           timedelta64[ns] 06:00:00
    isobaricInhPa  int64 500
    latitude       float64 30.5
    longitude      float64 275.5
    valid_time     datetime64[ns] 2021-10-01


Unfortunately, xarray itself doesn't currently have the ability to convert from one unit to another. Instead, an external package such as pint-xarray is needed to do this. Alternatively, MetPy (which uses xarray to handle data) can convert between units without destroying the metadata. Judge for yourself as to whether you are OK with converting data yourself or want to use an external package to do it for you!